In [1]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Date, Float, ForeignKey
from sqlalchemy.orm import sessionmaker,declarative_base

from perfin.config import DB_URL, USERS

### define engine and metadata

In [2]:
engine = create_engine(DB_URL, echo=True)

# Create a base class for the model
Base = declarative_base()

# Create a Session class
Session = sessionmaker(bind=engine)

### Define table schema

In [3]:
try:
    class User(Base):
        __tablename__ = 'users'
        
        user_name = Column(String, primary_key=True)
        email = Column(String)
        name = Column(String)
        password = Column(String)

        def as_dict(self):
            return {c.name: getattr(self, c.name) for c in self.__table__.columns}


except Exception as e:
    print(f"Error for Users table: {e}")

try:
    class Transaction(Base):
        __tablename__ = 'transactions'
        
        id = Column(Integer, primary_key=True)
        date = Column(Date)
        day_index = Column(Integer)
        category = Column(String)
        subcategory = Column(String)
        text = Column(String)
        amount = Column(Float)
        balance = Column(Float)
        user_key = Column(String, ForeignKey('users.user_name'))

        def as_dict(self):
            return {c.name: getattr(self, c.name) for c in self.__table__.columns}



except Exception as e:
    print(f"Error for Transactions table: {e}")

### Create tables

In [4]:
# Create the table
Base.metadata.create_all(engine)

2023-10-02 23:55:05,203 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-02 23:55:05,203 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2023-10-02 23:55:05,204 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-02 23:55:05,206 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2023-10-02 23:55:05,206 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-02 23:55:05,207 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("transactions")
2023-10-02 23:55:05,208 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-02 23:55:05,208 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("transactions")
2023-10-02 23:55:05,209 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-02 23:55:05,210 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	user_name VARCHAR NOT NULL, 
	email VARCHAR, 
	name VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (user_name)
)


2023-10-02 23:55:05,211 INFO sqlalchemy.engine.Engine [no key 0.00048s] ()
2023-10-02 23:55:05,213 INFO sqlalchemy.

### Create users

In [5]:
with Session() as session:
    users = [User(**user) for user in USERS]
    session.add_all(users)
    session.commit()

2023-10-02 23:55:06,985 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-02 23:55:06,987 INFO sqlalchemy.engine.Engine INSERT INTO users (user_name, email, name, password) VALUES (?, ?, ?, ?)
2023-10-02 23:55:06,988 INFO sqlalchemy.engine.Engine [generated in 0.00108s] [('admin', 'nicolaibthomsen@gmail.com', 'Nicolai', '$2b$12$XraL69WIGv9EQogv05T61Opoj.h7rSHV3bumDAlY7MpPM4ZdBCKJu'), ('cbrandborg', 'cbrandborg@hotmail.dk', 'Christian', '$2b$12$5ZK4c96UuM3fm37F38LUNeWxuDtyNBpbVJ4FBCfWZFQRim1khEaNa')]
2023-10-02 23:55:06,990 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
with Session() as session:
    users = session.query(User).all()


In [19]:
Transaction

TypeError: Transaction.as_dict() missing 1 required positional argument: 'self'

### Load data and build pipeline

In [ ]:
def read_transactions_from_csv(file_path: str) -> pd.DataFrame:
    return pd.read_csv(
        file_path, 
        sep=";", 
        parse_dates=["Dato"],
        date_format="%d.%m.%Y",
        encoding="ISO-8859-1",
        thousands=".",
        decimal=",",
        dtype={
            "Tekst": "string",
            "Kategori": "string",
            "Underkategori": "string",
            "Beløb": "float64",
            "Saldo": "float64",
        },
        usecols=[
            "Dato",
            "Tekst",
            "Kategori",
            "Underkategori",
            "Beløb",
            "Saldo"
        ],
    )

In [ ]:
def process_for_db(data, user) -> pd.DataFrame:
    """ Process the data for the database """

    return (
        data
        .reset_index(drop=False) # For primary key
        .rename(columns={
                "index": "id",
                "Dato": "date",
                "Tekst": "text",
                "Kategori": "category",
                "Underkategori": "subcategory",
                "Beløb": "amount",
                "Saldo": "balance",
            }
        )
        .assign(day_index=lambda d: d.groupby("date").cumcount())
        .astype(
            {"category": "category",
             "text": "string",
             "subcategory": "category",
             "date": "datetime64[ns]",
            }
        )
        .assign(
            user_key=user,
            text=lambda d: d.text.str.strip(),
            category=lambda d: d.category.str.strip(),
            subcategory=lambda d: d.subcategory.str.strip()
        )
    )

In [ ]:
data = (
    read_transactions_from_csv("db/transactions23.csv")
    .pipe(process_for_db, user=USERS[0]["user_name"])
)

### Add transactions to DB

In [ ]:
with Session() as session:
    transactions = [Transaction(**row) for row in data.to_dict(orient="records")]
    session.add_all(transactions)
    session.commit()

In [12]:
with Session() as session:
    data = pd.read_sql_table("transactions", session.bind)

2023-10-03 00:07:46,022 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-03 00:07:46,023 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("transactions")
2023-10-03 00:07:46,024 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-03 00:07:46,025 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-10-03 00:07:46,026 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-03 00:07:46,026 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-10-03 00:07:46,027 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-03 00:07:46,027 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("transactions")
2023-10-03 00:07:46,028 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-03 00:07:46,028 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name 

In [11]:
data

NameError: name 'data' is not defined

In [10]:
{
    'usernames': { 
        
        'admin': {
            'email': 'nicolaibthomsen@gmail.com',
            'name': 'Nicolai',
            'password': '$2b$12$XraL69WIGv9EQogv05T61Opoj.h7rSHV3bumDAlY7MpPM4ZdBCKJu'
        },
    
        'cbrandborg': {
            'email': 'cbrandborg@hotmail.dk',
            'name': 'Christian',
            'password': '$2b$12$5ZK4c96UuM3fm37F38LUNeWxuDtyNBpbVJ4FBCfWZFQRim1khEaNa'
        }
    }
}

{'usernames': {'admin': {'email': 'nicolaibthomsen@gmail.com',
   'name': 'Nicolai',
   'password': '$2b$12$XraL69WIGv9EQogv05T61Opoj.h7rSHV3bumDAlY7MpPM4ZdBCKJu'},
  'cbrandborg': {'email': 'cbrandborg@hotmail.dk',
   'name': 'Christian',
   'password': '$2b$12$5ZK4c96UuM3fm37F38LUNeWxuDtyNBpbVJ4FBCfWZFQRim1khEaNa'}}}